In [ ]:
[toc]

# Tensorflow2 利用 lstm 实现股票yuc

## 导入数据

In [17]:
import tensorflow as tf
import numpy as np
import tushare as ts
import pandas as pd
from sklearn.preprocessing import StandardScaler

seq_length = 20
input_dim = 4
output_dim = 1

stock_data = ts.get_k_data('600000', start='2016-01-01', end='2018-11-20')
features = ['open', 'high', 'low', 'volume']
target = ['close']
data = stock_data[features].values # shape = (685, 4)
target = stock_data[target].values # shape = (685, 1)

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2


## 划分验证集

这里不用 sklearn.model_selection.train_test_split，因为这会导致数据穿越

In [3]:
val_rate = 0.1
sample_size = data.shape[0]
train_size = int(sample_size * (1 - val_rate))
x_train, x_val, y_train, y_val = data[:train_size], data[train_size:], target[:train_size], target[:train_size]

print(f"train size: {train_size}, val sizse: {sample_size - train_size}")

train size: 616, val sizse: 69


## 对数据进行标准化

In [ ]:
x_scaler = StandardScaler()
y_scaler = StandardScaler()

x_train_scaled = x_scaler.fit_tr----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ansform(x_train)
y_train_scaled = y_scaler.fit_transform(y_train)
x_val_scaled = x_scaler.transform(x_val)
y_val_scaled = y_scaler.transform(y_val)

## 取得序列数据

In [19]:
def get_sequence_data(x, y):
    n = x_train.shape[0]
    sample_size = n - (seq_length + 1)
    x_sequence_sample = np.array([x_train[i:i+seq_length] for i in range(sample_size)])
    y_sequence_sample = np.array([y_train[i+seq_length] for i in range(sample_size)])
    return x_sequence_sample, y_sequence_sample

In [18]:
x_train_sequence, y_train_sequence = get_sequence_data(x_train_scaled, y_train_scaled)
x_val_sequence, y_val_sequence = get_sequence_data(x_val_scaled, y_val_scaled)

print(f"x_train_sequence shape: {x_train_sequence.shape}, y_train_sequence shape: {y_train_sequence.shape}")
print(f"x_val_sequence shape: {x_val_sequence.shape}, y_val_sequence shape: {y_val_sequence.shape}")

x_train_sequence shape: (595, 20, 4), y_train_sequence shape: (595, 1)
x_val_sequence shape: (595, 20, 4), y_val_sequence shape: (595, 1)


## 搭建网络

In [21]:
model.compile?

In [ ]:
# from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.models import Model

# 构建神经网络层 1层LSTM层+3层Dense层
lstm_input = Input(shape=(seq_length, input_dim), name='lstm_input') # input层不包括batch的shape
lstm_output = LSTM(128, activation='tanh', dropout=0.5)(lstm_input)  # LSTM网络
Dense_output_1 = Dense(64, activation='relu')(lstm_output)  #全连接网络
Dense_output_2 = Dense(16, activation='relu')(Dense_output_1)  #全连接网络
predictions = Dense(output_dim, activation='tanh')(Dense_output_2)  #全连接网络
model = Model(inputs=lstm_input, outputs=predictions)

model.compile(optimizer=tf.keras.optimizers.SGD(1), loss='mse', metrics=['mse'])
model.fit(x_train_sequence, y_train_sequence, validation_data=(x_val_sequence, y_val_sequence), batch_size=1, epochs=100)

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tushare as ts
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

#参数设置/parameter setting
timesteps = seq_length = 20  #时间窗/window length
data_dim = 5  #输入数据维度/dimension of input data
output_dim = 1  #输出数据维度/dimension of output data

stock_data = ts.get_k_data('600000', start='2016-01-01', end='2018-11-20')
xy = stock_data[['open', 'close', 'high', 'low', 'volume']] #变量选取Open,High,Low,Close,Volume
xy = xy.values

#切分训练集合测试集/split to train and testing
train_size = int(len(xy) * 0.7)
test_size = len(xy) - train_size
xy_train, xy_test = np.array(xy[0:train_size]), np.array(
    xy[train_size:len(xy)])

#对training set进行预处理
scaler = MinMaxScaler()
xy_train_new = scaler.fit_transform(xy_train)
x_new = xy_train_new[:, 0:5]
y_new = xy_train_new[:, 1]

x = x_new
y = y_new
dataX = []
dataY = []
for i in range(0, len(y) - seq_length):
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price
    print(_x, "->", _y)
    dataX.append(_x)
    dataY.append(_y)

#处理数据shape,准备进入神经网络层
x_real = np.vstack(dataX).reshape(-1, seq_length, data_dim)
y_real = np.vstack(dataY).reshape(-1, output_dim)
print(x_real.shape)
print(y_real.shape)
dataX = x_real
dataY = y_real

trainX, trainY = dataX, dataY

#对test set进行预处理，这里用了training的scaler
xy_test_new = scaler.transform(xy_test)
x_new = xy_test_new[:, 0:5]
y_new = xy_test_new[:, 1]

x = x_new
y = y_new
dataX = []
dataY = []
for i in range(0, len(y) - seq_length):
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next price change
    print(_x, "->", _y)
    dataX.append(_x)
    dataY.append(_y)

#处理数据shape,准备进入神经网络层
x_real = np.vstack(dataX).reshape(-1, seq_length, data_dim)
y_real = np.vstack(dataY).reshape(-1, output_dim)
print(x_real.shape)
print(y_real.shape)
dataX = x_real
dataY = y_real

testX, testY = dataX, dataY

本接口即将停止更新，请尽快使用Pro版接口：https://tushare.pro/document/2
[[0.45944897 0.30630631 0.38223801 0.38939607 0.15973704]
 [0.26076834 0.36373874 0.330373   0.35428371 0.23350485]
 [0.3612728  0.39864865 0.34920071 0.42205056 0.1808774 ]
 [0.33566162 0.2515015  0.29236234 0.35428371 0.01564396]
 [0.3201397  0.24662162 0.28774423 0.24016854 0.29817373]
 [0.18354676 0.13701201 0.18650089 0.25175562 0.38334576]
 [0.17345751 0.23686186 0.20284192 0.27984551 0.2210021 ]
 [0.27629026 0.22184685 0.29236234 0.34269663 0.18599316]
 [0.12960807 0.35397898 0.31367673 0.23560393 0.21850369]
 [0.33566162 0.26914414 0.35133215 0.3823736  0.1806465 ]
 [0.24020179 0.29391892 0.31367673 0.33567416 0.11536841]
 [0.330617   0.31644144 0.28774423 0.37780899 0.10173867]
 [0.28676756 0.18693694 0.24547069 0.27984551 0.13048013]
 [0.1862631  0.01989489 0.18863233 0.14466292 0.12221664]
 [0.08847497 0.1295045  0.1062167  0.14220506 0.15864877]
 [0.12960807 0.13213213 0.12042629 0.19592697 0.07259278]
 [0.11913077 0.0397

 [0.58634071 0.56906907 0.52220249 0.66116573 0.01913081]] -> 0.5608108108108105
[[0.61816065 0.61148649 0.56767318 0.68223315 0.00282773]
 [0.65618937 0.58033033 0.5705151  0.68504213 0.00082798]
 [0.63562282 0.58896396 0.55168739 0.69557584 0.02546499]
 [0.60651921 0.59159159 0.5410302  0.6875     0.03154494]
 [0.60069849 0.59159159 0.52753108 0.66397472 0.01512338]
 [0.61544432 0.60585586 0.54884547 0.67696629 0.01584267]
 [0.6387272  0.62274775 0.56767318 0.70610955 0.02793447]
 [0.67365153 0.64264264 0.58365897 0.72717697 0.05595804]
 [0.6709352  0.61411411 0.57833037 0.71664326 0.02327912]
 [0.65618937 0.63963964 0.59182948 0.71910112 0.08252436]
 [0.67365153 0.62274775 0.58650089 0.72191011 0.03886197]
 [0.64454792 0.58333333 0.56234458 0.67696629 0.02736725]
 [0.59798215 0.65953453 0.60497336 0.68504213 0.10240479]
 [0.67365153 0.6509009  0.63161634 0.73490169 0.04437236]
 [0.67365153 0.61411411 0.58365897 0.71383427 0.04214777]
 [0.64454792 0.59722222 0.56483126 0.69030899 0.0

(459, 20, 5)
(459, 1)
[[0.69654637 0.72522523 0.669627   0.76369382 0.24873925]
 [0.73108265 0.68055556 0.6625222  0.73946629 0.31204837]
 [0.70818782 0.75487988 0.78898757 0.77773876 0.5557325 ]
 [0.79627474 0.80668168 0.72930728 0.83321629 0.48331418]
 [0.86922778 0.80668168 0.85612789 0.87851124 0.76338567]
 [0.71594878 0.63213213 0.69413854 0.71523876 1.23388922]
 [0.64648816 0.68055556 0.61349911 0.70470506 0.97879434]
 [0.73108265 0.92942943 0.9928952  0.77773876 1.73366275]
 [0.94994179 0.95908408 0.90870337 0.94101124 0.78829882]
 [0.99223904 0.87012012 0.89449378 0.94101124 0.98155676]
 [0.92704695 0.88851351 0.8277087  0.92731742 0.68004779]
 [0.91540551 0.75863363 0.81385435 0.84726124 0.46556972]
 [0.76173846 0.78078078 0.70834813 0.80898876 0.51174638]
 [0.79627474 0.8292042  0.75737123 0.79494382 0.77236285]
 [0.81567714 0.76951952 0.70834813 0.81952247 0.29760697]
 [0.7427241  0.89977477 0.8312611  0.80898876 0.66309865]
 [0.85409391 0.89602102 0.81740675 0.88553371 1.10

In [7]:
model.fit?

In [8]:
from tensorflow.keras.layers import Input, Dense, LSTM
from tensorflow.keras.models import Model

# 构建神经网络层 1层LSTM层+3层Dense层
lstm_input = Input(
    shape=(seq_length, data_dim), name='lstm_input'
)  #shape: 形状元组（整型）不包括batch size。表示了预期的输入将是一批（seq_len,data_dim）的向量。
lstm_output = LSTM(128, activation='tanh', dropout=0.5)(lstm_input)  #LSTM网络
#units: Positive integer,dimensionality of the output space.
#dropout: Float between 0 and 1. Fraction of the units to drop for the linear transformation of the inputs.
Dense_output_1 = Dense(64, activation='relu')(lstm_output)  #全连接网络
Dense_output_2 = Dense(16, activation='relu')(Dense_output_1)  #全连接网络
predictions = Dense(output_dim, activation='tanh')(Dense_output_2)  #全连接网络
model = Model(inputs=lstm_input, outputs=predictions)

model.compile(optimizer='adam', loss='mse', metrics=['mse'])
#Configures the model for training.
#optimizer: String (name of optimizer) or optimizer instance. See optimizers.
#loss: String (name of objective function) or objective function.The loss value will be minimized by the model.
#metrics: List of metrics to be evaluated by the model during training and testing. Typically you will use  metrics=['accuracy'].
model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=32, epochs=100)

Train on 459 samples, validate on 186 samples
Epoch 1/100
459/459 [==============================] - 2s 3ms/sample - loss: 0.0478 - mse: 0.0478 - val_loss: 0.1092 - val_mse: 0.1092
Epoch 2/100
459/459 [==============================] - 0s 401us/sample - loss: 0.0123 - mse: 0.0123 - val_loss: 0.1104 - val_mse: 0.1104
Epoch 3/100
459/459 [==============================] - 0s 386us/sample - loss: 0.0129 - mse: 0.0129 - val_loss: 0.1076 - val_mse: 0.1076
Epoch 4/100
459/459 [==============================] - 0s 387us/sample - loss: 0.0106 - mse: 0.0106 - val_loss: 0.1033 - val_mse: 0.1033
Epoch 5/100
459/459 [==============================] - 0s 433us/sample - loss: 0.0105 - mse: 0.0105 - val_loss: 0.0991 - val_mse: 0.0991
Epoch 6/100
459/459 [==============================] - 0s 424us/sample - loss: 0.0105 - mse: 0.0105 - val_loss: 0.0961 - val_mse: 0.0961
Epoch 7/100
459/459 [==============================] - 0s 387us/sample - loss: 0.0108 - mse: 0.0108 - val_loss: 0.0919 - val_mse: 0.09

In [9]:
# 保存模型
model.save('model.h5')  # HDF5文件，pip install h5py

trainPredict = model.predict(trainX)
trainPredict1 = scaler.inverse_transform(trainPredict)
trainY1 = scaler.inverse_transform(trainY)
plt.plot(trainY1, color='blue')
plt.plot(trainPredict1, color='orange')
plt.show()
print(max(abs(trainPredict1 - trainY1)))

# 载入模型
from tensorflow.keras.models import load_model
model = load_model('model.h5')

testPredict = model.predict(testX)
testPredict1 = testPredict * scaler.data_range_[1] + scaler.data_min_[1]
testY1 = testY * scaler.data_range_[1] + scaler.data_min_[1]
plt.plot(testY1, color='blue')
plt.plot(testPredict1, color='orange')
plt.show()
print(max(abs(testPredict1 - testY1)))

ValueError: non-broadcastable output operand with shape (459,1) doesn't match the broadcast shape (459,5)

In [2]:
testY.shape

(186, 1)

In [3]:
testX.shape

(186, 20, 5)